<a href="https://colab.research.google.com/github/Mgiri1234/LabProject_VLM/blob/main/VLM_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q -U bitsandbytes
%pip install transformers datasets torchvision torch
%pip install -q pytorch_lightning  # For training


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which

In [2]:
pip install bitsandbytes accelerate transformers

In [3]:
# Install dependencies for quantized model usage
# %pip install bitsandbytes accelerate transformers

from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, BitsAndBytesConfig
from PIL import Image
import requests
import torch

# Model details
model_id = "google/paligemma-3b-mix-224"

# Load image from URL
url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)

# Configuration for 8-bit quantization (use 4-bit for even lower memory usage if desired)
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load the model with quantization config for CPU
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=quantization_config
).eval()

# Load processor for preparing inputs
processor = AutoProcessor.from_pretrained(model_id)

# Create prompt (in Spanish for this example)
prompt = "what objects are there in the image"
model_inputs = processor(text=prompt, images=image, return_tensors="pt")

# Generate caption
input_len = model_inputs["input_ids"].shape[-1]

with torch.no_grad():
    generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
    generation = generation[0][input_len:]
    decoded = processor.decode(generation, skip_special_tokens=True)

print("Generated caption:", decoded)


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1885: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Generated caption: Car, Land vehicle, Vehicle, Wheel.


In [4]:
pip install datasets transformers bitsandbytes accelerate pillow


In [6]:
import torch
from datasets import load_dataset
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, BitsAndBytesConfig
from PIL import Image

# Load OpenImagesV7 dataset (segmentation task)
dataset = load_dataset("dalle-mini/open-images", split="validation")

# Model configuration
model_id = "google/paligemma-3b-mix-224"
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Load PaliGemma model with 8-bit precision
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id, quantization_config=quantization_config
).eval()

# Load the processor
processor = AutoProcessor.from_pretrained(model_id)

# Check if CUDA is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Function to load and process image from dataset


Resolving data files:   0%|          | 0/3606 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

KeyError: 'image'

In [38]:
def process_image(sample):
    # Load the image
    # print("hello")
    image_url = sample['url']  # OpenImagesV7 provides URLs for images
    image = Image.open(requests.get(image_url, stream=True).raw)

    # Define the segmentation task prompt
    prompt = "describe the image"

    # Process the image and prompt
    model_inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)

    # Get the input length for text generation
    input_len = model_inputs["input_ids"].shape[-1]

    # Generate the output
    with torch.no_grad():
        generation = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
        generation = generation[0][input_len:]

    # Decode the result (segmentation output)
    decoded_output = processor.decode(generation, skip_special_tokens=True)
    return decoded_output

# Example usage: process the first image in the dataset
sample_image = dataset[1]
segmentation_result = process_image(sample_image)
print(segmentation_result)


In this image we can see a glass with drink, straw, food item and a person.


In [43]:
print(dataset[0])

{'url': 'https://c4.staticflickr.com/6/5742/20795376805_903e9cc61d_o.jpg', 'key': 13, 'shard_id': 0, 'status': 'success', 'error_message': None, 'width': 640.0, 'height': 400.0, 'exif': '{}', 'original_width': 640.0, 'original_height': 400.0}


In [40]:
from transformers import PaliGemmaForConditionalGeneration, PaliGemmaProcessor
import torch

# Load the processor and model with pre-trained weights
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=quantization_config
).eval()

# Load processor for preparing inputs
processor = AutoProcessor.from_pretrained(model_id)

# Optimizer and Scheduler (for learning rate)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)


/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [49]:
model.config.num_channels=3

In [41]:
from torch.utils.data import random_split

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1*len(dataset))
test_size = int(0.1*len(dataset))
train_dataset, val_dataset,test_dataset = random_split(dataset, [train_size, val_size,test_size])

In [55]:
from torch.utils.data import DataLoader
from PIL import Image
import requests
from io import BytesIO

# Define your prompt here
prompt = "What is in this image?"

def load_image_from_url(url):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Check for HTTP errors
        return Image.open(BytesIO(response.content))
    except Exception as e:
        print(f"Error loading image from {url}: {e}")
        return None  # Return None for failed image loads

def collate_fn(batch):
    images = []
    for item in batch:
        img = load_image_from_url(item['url'])
        if img is not None:
            images.append(img)

    if len(images) == 0:
        raise ValueError("All images in the batch failed to load.")

    # Process the batch with the same prompt for all images
    # Add do_convert_rgb=True to handle potential single-channel images
    inputs = processor(images=images, text=[prompt] * len(images), return_tensors="pt", padding=True, do_convert_rgb=True)
    return inputs

# Assuming `train_dataset` contains your dataset with image URLs
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)


In [56]:
torch.cuda.empty_cache()

In [57]:
from tqdm import tqdm
num_epochs = 100

# Training loop
model.train()
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    for batch in tqdm(train_loader):
        # Move inputs to the correct device (GPU/CPU)
        input_ids = batch['input_ids'].to(device)
        pixel_values = batch['pixel_values'].to(device)

        # Ensure attention_mask exists and is on the correct device
        if 'attention_mask' in batch:
            attention_mask = batch['attention_mask'].to(device)
        else:
            # Create an attention mask if it doesn't exist
            attention_mask = torch.ones_like(input_ids).to(device)

        # Forward pass (with image-text inputs)
        outputs = model(input_ids=input_ids, pixel_values=pixel_values, labels=input_ids, attention_mask=attention_mask) # Pass attention_mask to the model
        loss = outputs.loss

        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Log the loss (optional)
        print(f"Loss: {loss.item()}")

Epoch 1/100


  0%|          | 0/4162 [00:00<?, ?it/s]

Error loading image from https://c7.staticflickr.com/4/3084/2808129544_587f092ef3_o.jpg: 410 Client Error: Gone for url: https://c7.staticflickr.com/4/3084/2808129544_587f092ef3_o.jpg
Error loading image from https://farm8.staticflickr.com/7156/6777241443_81962b95e8_o.jpg: 404 Client Error: Not Found for url: https://farm8.staticflickr.com/7156/6777241443_81962b95e8_o.jpg


  0%|          | 0/4162 [00:05<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 26.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 5.06 MiB is free. Process 7262 has 14.74 GiB memory in use. Of the allocated memory 14.40 GiB is allocated by PyTorch, and 204.20 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)